In [1]:
import torch
import sys
import os
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import MLFlowLogger

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))


if project_root not in sys.path:
    sys.path.append(project_root)
    
from model_utils import PretrainedEfficientNetV2, ClassificationData, no_augmentation

torch.cuda.empty_cache()
torch.set_float32_matmul_precision("medium")
L.seed_everything(111)

Seed set to 111


111

In [2]:
model = PretrainedEfficientNetV2(batch_size=64)
data = ClassificationData(batch_size=64)
logger = MLFlowLogger(experiment_name="EfficentNetV2", save_dir="mlruns")
trainer = L.Trainer(max_epochs=20, 
                    logger=logger, 
                    callbacks=[ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/efficientnet_v2", filename="augmentation-{epoch:02d}-{val_f1_macro:.2f}"), 
                               EarlyStopping(monitor="train_loss", min_delta=0.01, patience=3, mode="min", verbose=True)], 
                    precision="16-mixed", 
                    num_sanity_val_steps=0)
trainer.fit(model, datamodule=data)
trainer.test(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\SPCX\Desktop\github-repositories\dl-cnn\.venv\Lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\SPCX\Desktop\github-repositories\dl-cnn\model\checkpoints\efficientnet_v2 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | EfficientNet     | 20.2 M | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | val_metrics   | MetricCollection | 0      | train
4 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.761    Total e

Epoch 0: 100%|██████████| 1407/1407 [01:48<00:00, 12.92it/s, v_num=c56b]

Metric train_loss improved. New best score: 1.456


Epoch 1: 100%|██████████| 1407/1407 [01:32<00:00, 15.27it/s, v_num=c56b]

Metric train_loss improved by 0.208 >= min_delta = 0.01. New best score: 1.248


...

Monitored metric train_loss did not improve in the last 3 records. Best score: 1.248. Signaling Trainer to stop.


Epoch 4: 100%|██████████| 1407/1407 [01:32<00:00, 15.23it/s, v_num=c56b]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\SPCX\Desktop\github-repositories\dl-cnn\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:420: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


Testing DataLoader 0: 100%|██████████| 1407/1407 [00:19<00:00, 73.97it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.8110043406486511
      test_f1_macro         0.3092794716358185
     test_precision         0.34225529432296753
       test_recall          0.33291110396385193
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_auroc': 0.8110043406486511,
  'test_f1_macro': 0.3092794716358185,
  'test_precision': 0.34225529432296753,
  'test_recall': 0.33291110396385193}]

In [3]:
model = PretrainedEfficientNetV2(batch_size=64)
data = ClassificationData(batch_size=64, transform=no_augmentation)
logger = MLFlowLogger(experiment_name="EfficentNetV2-no-augmentation", save_dir="mlruns")
trainer = L.Trainer(max_epochs=20, 
                    logger=logger, 
                    callbacks=[ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/efficientnet_v2", filename="no-augmentation-{epoch:02d}-{val_f1_macro:.2f}"), 
                               EarlyStopping(monitor="train_loss", min_delta=0.01, patience=3, mode="min", verbose=True)], 
                    precision="16-mixed", 
                    num_sanity_val_steps=0)
trainer.fit(model, datamodule=data)
trainer.test(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Experiment with name EfficentNetV2-no-augmentation not found. Creating it.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | EfficientNet     | 20.2 M | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | val_metrics   | MetricCollection | 0      | train
4 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.761    Total estimated model params size (MB)
730       Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 1407/1407 [01:46<00:00, 13.15it/s, v_num=2d51]

Metric train_loss improved. New best score: 1.172


Epoch 1: 100%|██████████| 1407/1407 [01:28<00:00, 15.83it/s, v_num=2d51]

Metric train_loss improved by 0.298 >= min_delta = 0.01. New best score: 0.874


...

Monitored metric train_loss did not improve in the last 3 records. Best score: 0.438. Signaling Trainer to stop.


Epoch 12: 100%|██████████| 1407/1407 [01:30<00:00, 15.56it/s, v_num=2d51]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 1407/1407 [00:18<00:00, 77.21it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.9335421919822693
      test_f1_macro         0.6256684064865112
     test_precision         0.6376968026161194
       test_recall          0.6314777135848999
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_auroc': 0.9335421919822693,
  'test_f1_macro': 0.6256684064865112,
  'test_precision': 0.6376968026161194,
  'test_recall': 0.6314777135848999}]

### Larger batch size with no augmentation

In [4]:
model = PretrainedEfficientNetV2(batch_size=512)
data = ClassificationData(batch_size=512, transform=no_augmentation)
logger = MLFlowLogger(experiment_name="EfficentNetV2-no-augmentation", save_dir="mlruns")
trainer = L.Trainer(max_epochs=30, 
                    logger=logger, 
                    callbacks=[ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/efficientnet_v2", filename="no-augmentation_bs-512_{epoch:02d}-{val_f1_macro:.2f}"), 
                               EarlyStopping(monitor="train_loss", min_delta=0.005, patience=3, mode="min", verbose=True)], 
                    precision="16-mixed", 
                    num_sanity_val_steps=0)
trainer.fit(model, datamodule=data)
trainer.test(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | EfficientNet     | 20.2 M | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | val_metrics   | MetricCollection | 0      | train
4 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.761    Total estimated model params size (MB)
730       Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 176/176 [00:44<00:00,  3.92it/s, v_num=a2e4]

Metric train_loss improved. New best score: 1.271


Epoch 1: 100%|██████████| 176/176 [00:27<00:00,  6.40it/s, v_num=a2e4]

Metric train_loss improved by 0.481 >= min_delta = 0.005. New best score: 0.790


...

Monitored metric train_loss did not improve in the last 3 records. Best score: 0.126. Signaling Trainer to stop.


Epoch 20: 100%|██████████| 176/176 [00:27<00:00,  6.41it/s, v_num=a2e4]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 176/176 [00:09<00:00, 17.88it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.9565265774726868
      test_f1_macro         0.7228973507881165
     test_precision          0.727575421333313
       test_recall          0.7230111360549927
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_auroc': 0.9565265774726868,
  'test_f1_macro': 0.7228973507881165,
  'test_precision': 0.727575421333313,
  'test_recall': 0.7230111360549927}]

In [5]:
model = PretrainedEfficientNetV2(batch_size=1024)
data = ClassificationData(batch_size=1024, transform=no_augmentation)
logger = MLFlowLogger(experiment_name="EfficentNetV2-no-augmentation", save_dir="mlruns")
trainer = L.Trainer(max_epochs=30, 
                    logger=logger, 
                    callbacks=[ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/efficientnet_v2", filename="no-augmentation_bs-1024_{epoch:02d}-{val_f1_macro:.2f}"), 
                               EarlyStopping(monitor="train_loss", min_delta=0.005, patience=3, mode="min", verbose=True)], 
                    precision="16-mixed", 
                    num_sanity_val_steps=0)
trainer.fit(model, datamodule=data)
trainer.test(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | EfficientNet     | 20.2 M | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | val_metrics   | MetricCollection | 0      | train
4 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.761    Total estimated model params size (MB)
730       Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 88/88 [00:42<00:00,  2.08it/s, v_num=29c6]

Metric train_loss improved. New best score: 1.431


Epoch 1: 100%|██████████| 88/88 [00:25<00:00,  3.49it/s, v_num=29c6]

Metric train_loss improved by 0.564 >= min_delta = 0.005. New best score: 0.866


...

Monitored metric train_loss did not improve in the last 3 records. Best score: 0.105. Signaling Trainer to stop.


Epoch 20: 100%|██████████| 88/88 [00:25<00:00,  3.49it/s, v_num=29c6]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 88/88 [00:09<00:00,  9.30it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.9559515118598938
      test_f1_macro         0.7169007658958435
     test_precision         0.7209777235984802
       test_recall          0.7179888486862183
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_auroc': 0.9559515118598938,
  'test_f1_macro': 0.7169007658958435,
  'test_precision': 0.7209777235984802,
  'test_recall': 0.7179888486862183}]

In [6]:
model = PretrainedEfficientNetV2(batch_size=256)
data = ClassificationData(batch_size=256, transform=no_augmentation)
logger = MLFlowLogger(experiment_name="EfficentNetV2-no-augmentation", save_dir="mlruns")
trainer = L.Trainer(max_epochs=30, 
                    logger=logger, 
                    callbacks=[ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/efficientnet_v2", filename="no-augmentation_bs-256_{epoch:02d}-{val_f1_macro:.2f}"), 
                               EarlyStopping(monitor="train_loss", min_delta=0.005, patience=3, mode="min", verbose=True)], 
                    precision="16-mixed", 
                    num_sanity_val_steps=0)
trainer.fit(model, datamodule=data)
trainer.test(model, datamodule=data)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | EfficientNet     | 20.2 M | train
1 | loss_fn       | CrossEntropyLoss | 0      | train
2 | train_metrics | MetricCollection | 0      | train
3 | val_metrics   | MetricCollection | 0      | train
4 | test_metrics  | MetricCollection | 0      | train
-----------------------------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.761    Total estimated model params size (MB)
730       Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 352/352 [00:50<00:00,  6.95it/s, v_num=e217]

Metric train_loss improved. New best score: 1.171


Epoch 1: 100%|██████████| 352/352 [00:33<00:00, 10.39it/s, v_num=e217]

Metric train_loss improved by 0.417 >= min_delta = 0.005. New best score: 0.754


...

Monitored metric train_loss did not improve in the last 3 records. Best score: 0.168. Signaling Trainer to stop.


Epoch 17: 100%|██████████| 352/352 [00:33<00:00, 10.37it/s, v_num=e217]


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 352/352 [00:10<00:00, 33.56it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       test_auroc           0.9572105407714844
      test_f1_macro         0.7215771675109863
     test_precision         0.7270466089248657
       test_recall          0.7209555506706238
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_auroc': 0.9572105407714844,
  'test_f1_macro': 0.7215771675109863,
  'test_precision': 0.7270466089248657,
  'test_recall': 0.7209555506706238}]